# Neural Style Transfer
이번 시간에는 사진에 특정 스타일을 투영하는 **Neural Style Transfer**에 대해 배워보겠습니다. 아래와 같은 결과물을 보실 수 있을 겁니다! (아마도?)

이 Neural Style Transfer 기술은 2015년에 *Leon A. Gatys*에 의해 처음 발표되었는데요, 무려 박사 졸업 논문인데 인용횟수 7300+... 나도 박사과정인데... 한없이 작아진다.
+

## Implementation

### Line-by-Line with TensorFlow

In [3]:
import tensorflow as tf

import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

ModuleNotFoundError: No module named 'tensorflow'

### Few Lines of shortcuts with TensorFlow-Hub

# Reference
### Papers
+ dd

### Source Code
+ ss